In [0]:
# TensorFlow Tutorials
# ML basics with Keras 
# Keras Tuner
# https://www.tensorflow.org/tutorials/keras/keras_tuner

# Keras tuner: https://keras-team.github.io/keras-tuner/
# Keras tuner blog: https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html
# Hyperband: https://arxiv.org/pdf/1603.06560.pdf

# tfds reshufle: https://www.tensorflow.org/api_docs/python/tf/reshape
# TF.data.Dataset: https://www.tensorflow.org/api_docs/python/tf/data/Dataset
# TF Keras: https://www.tensorflow.org/guide/keras

# Using the SavedModel format: https://www.tensorflow.org/guide/saved_model#save_and_restore_variables
# Writing custome layers and models in keras: https://www.tensorflow.org/guide/keras/custom_layers_and_models
# HDF5: https://en.wikipedia.org/wiki/Hierarchical_Data_Format

# Keras get file: https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file
# pd map: reassign values in a series: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html
# pd get_dummies  to one-hot encode a categorical column: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
# pd pop: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pop.html
# TF Keras EarlyStopping: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
# np expand dimension: https://docs.scipy.org/doc/numpy/reference/generated/numpy.expand_dims.html
# np collapse dimension: https://docs.scipy.org/doc/numpy/reference/generated/numpy.squeeze.html


# https://www.tensorflow.org/tutorials/structured_data/feature_columns
# https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

# mnist tfds dataset: https://www.tensorflow.org/datasets/catalog/mnist

# tfds.load: https://www.tensorflow.org/datasets/api_docs/python/tfds/load
# TF Hub (Transfer Learning): https://www.tensorflow.org/hub

# TF Keras Embedding: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
# TF Hub Keraslayer: https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer
# TF Keras regularizers: https://keras.io/api/layers/regularizers/    https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/Regularizer
# TF Keras Cnv1D: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D
# TF Keras LSTM: https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM
# TF Keras Dense: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
# TF Keras activation functions: https://keras.io/api/layers/activations/

# Live training plots: https://gist.github.com/stared/dfb4dfaf6d9a8501cd1cc8b8cb806d2e


# Required to save models in HDF5 format
!pip install -q pyyaml h5py



import pathlib
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import IPython


import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

from IPython import display
from IPython.display import clear_output
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from datetime import *


!pip install -q -U keras-Tuner
import kerastuner as kt


# logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
# shutil.rmtree(logdir, ignore_errors=True)

# # Use seaborn for pairplot
# !pip install -q seaborn

# # Use some functions from tensorflow_docs
# !pip install -q git+https://github.com/tensorflow/docs

# import tensorflow_docs as tfdocs
# import tensorflow_docs.plots
# import tensorflow_docs.modeling



print("TF Version: ", tf.__version__)
# print("Eager mode: ", tf.executing_eagerly())
# print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

print('\n')


starttime = datetime.now()


# Download the dataset:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# # Take the first 1000 examples:
# train_labels = train_labels[:1000]
# test_labels = test_labels[:1000]

# print(train_images.shape)

# # reshape(-1) flattens the image data.
# train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
# test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

# train_images = train_images.take(1000)  #.reshape(-1, 28*28) / 255.0

train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

print(train_images.shape)



et1 = datetime.now() - starttime

print( '\n-> Elapsed execution time: %0.4f seconds.\n' %(et1.total_seconds()) )

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


2.2.0
     |████████████████████████████████| 61kB 1.8MB/s 
TF Version:  2.2.0
GPU is available


11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)

-> Elapsed execution time: 0.8102 seconds.



## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined `HyperModel` classes - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) and [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [0]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)  # integer
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])   # choice() is an inbuilt function in Python programming language that returns a random item from a list, tuple, or string.
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner. 

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`)

In [0]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')    

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.



Before running the hyperparameter search, define a callback to clear the training outputs at the end of every training step.

In [0]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [0]:
starttime = datetime.now()


tuner.search(train_images, train_labels, epochs = 10, validation_data = (test_images, test_labels), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


et1 = datetime.now() - starttime

print( '\n-> Elapsed execution time: %0.4f seconds.\n' %(et1.total_seconds()) )

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.


-> Elapsed execution time: 438.0620 seconds.



To finish this tutorial, retrain the model with the optimal hyperparameters from the search.

In [0]:
starttime = datetime.now()


# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(train_images, train_labels, epochs = 10, validation_data = (test_images, test_labels))


et1 = datetime.now() - starttime

print( '\n-> Elapsed execution time: %0.4f seconds.\n' %(et1.total_seconds()) )

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2009 - accuracy: 0.9410 - val_loss: 0.1075 - val_accuracy: 0.9649
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0799 - accuracy: 0.9762 - val_loss: 0.0907 - val_accuracy: 0.9715
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0523 - accuracy: 0.9835 - val_loss: 0.0692 - val_accuracy: 0.9792
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0348 - accuracy: 0.9888 - val_loss: 0.0690 - val_accuracy: 0.9802
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0288 - accuracy: 0.9906 - val_loss: 0.0688 - val_accuracy: 0.9795
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0202 - accuracy: 0.9934 - val_loss: 0.0677 - val_accuracy: 0.9813
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0185 - accuracy: 0.9933 - val_loss: 0.0676 - val_accuracy:

The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite = True` argument while instantiating the tuner.

In [0]:
!ls
print(os.listdir(os.getcwd()))

!ls my_dir
print(os.listdir('my_dir'))

!ls my_dir/intro_to_kt
print(os.listdir('my_dir/intro_to_kt'))

my_dir	sample_data
['.config', 'my_dir', 'sample_data']
intro_to_kt
['intro_to_kt']
oracle.json				trial_74e793cec9551e9b31d000f97eb706ef
trial_00efc3c1773bb05a0945ea3d6f0e2562	trial_78f17e53b3a4c200596c30533dd0cdb3
trial_0362abddd2772c852d21be660d9e1632	trial_89f4dd07f3a98042cd0b2ce1fba0b897
trial_0ebda25ec47cf6e8904e94fa40cb65b4	trial_8f9bddcf34ecce27c933ade34c25fe34
trial_0fc79fd6fdc6dae3e89a786fb16377da	trial_91bdd82b0a9081ce9d764b0211b15bc1
trial_270594691697be306ab0c9a1a66f66cd	trial_a2d8375206a51f8ca72026e9169532be
trial_2873e2a89b5c033d4e7f17cda5f7bc9c	trial_aa927269be380c89cb6a397273cfc824
trial_331203a387ca4d4c53dbcdd315e5db0c	trial_aaa7592e230b9555d5e51a6c39e1a540
trial_480ecfc216bac1502212304121536f48	trial_b7f7b8a13ad1620e4dfd0c3a5490508a
trial_4a2624920d60e7f96c089f81b044642b	trial_bb68b3357a004b5081d2d17cd91571c2
trial_4a74c86d5ff51cf1e0afb0781c804b0c	trial_c794a22f457f52332ed87eb6bf87b5cf
trial_4b5eff50efcf78e0a95e6dad19b45a5c	trial_c9491f16fab9909efd11fc5814e32511
tria

## Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

* [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

Also check out the [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) in TensorBoard to interactively tune your model hyperparameters.